# Training and Deploying Fraud Model
---

## Introduction

In this lab, we are going to train a Machine Learning (ML) model on the transformed dataset that was exported from Amazon SageMaker Canvas. We will first split the data into train, test, and validation data. Then we will train the model using train dataset, deploy it on a SageMaker endpoint, validate the model's accuracy using validation dataset, and check the results by testing with the test dataset.

This diagram summarizes the next steps:

![TrainDeployModel.png](images/train-deploy-model.png)

1. Read the dataset from S3 bucket to a Pandas dataframe object.
2. Transformed data is explored. 
3. Split the data into train/validation/test datasets.
4. Train the model using XGBoost algorithm.
5. Deploy the model endpoint
6. Validate the model, check for precision and accuracy using confusion matrix.
7. Test the model using sample transactions.

**NOTE: Please make sure you complete all the steps mentioned in the Clean-up section of this lab to avoid any charges due to the resources created in this lab.**


## Index

---

1. [Prerequisites](#Prerequisites)
    1. [Install required third party libraries](#install-required-third-party-libraries)
1. [Module 1: Prepare Train, Validation and Test Data](#module-1:-prepare-train,-validation-and-test-data)
    1. [Explore the dataset](#explore-the-dataset)
    1. [Prepare data for training and inference](#prepare-data-for-training-and-inference)
    1. [Upload the Train and Validation dataset to S3](#upload-the-train-and-validation-dataset-to-S3)
1. [Module 2: Train and deploy XGBoost model](#module-2:-train-and-deploy-xgboost-model)
    1. [Training: Create the estimator](#training:-create-the-estimator)
    1. [Set HyperParameters](#set-hyperParameters)
    1. [Start the training job](#start-the-training-job)
    1. [Deploy the model](#deploy-the-model)
1. [Module 3: Predict and evaluate model performance](#module-3:-predict-and-evaluate-model-performance)

## Prerequisites
---

### Install required third party libraries 
The default SageMaker notebook container already has many helpful packages like Pandas, Numpy and Matplotlib.

The following python modules will be installed in the next cell:
1. `boto3`: Python SDK to call the AWS API
2. `pip`: for package installations
4. `sagemaker`: Python SDK to call the SageMaker API
5. `s3fs`: for reading the files from S3 bucket

#### (if you encounter a warning about pip running as root user - don't worry about it. We're running everything in a container, so it's not going to break anything. You may also incur some incompatible library errors. Re-run cell 1 again to re-install the libraries and to get rid of them) 

In [ ]:
# Cell 1
# Install boto3
!pip install boto3 -q

In [ ]:
# Cell 2
# Install pip
!python -m pip install -Uq pip

In [ ]:
# Cell 3
# Install SageMaker
!pip install sagemaker -q

#### NOTE: You may incur in some incompatible library errors after running the step below. In that case, just re-run the cell again to re-install the libraries. If the error persists, just ignore and move to running the next cells.

In [ ]:
# Cell 4
# Install s3fs
!pip install s3fs -q


## Module 1: Prepare Train, Validation and Test Data
---

In this section, we will fetch the dataset from the S3 folder where you exported the transformed dataset via SageMaker Canvas. We will then use Pandas library to explore the data. 

In [ ]:
# Cell 5
# Import libraries, define role, session and AWS region.
import os
import boto3
import re
import sagemaker
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  # visualisation

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = boto3.Session().region_name

#### NOTE: In the cell below, replace the respective placeholder with the S3 URI of the dataset that was exported from Amazon SageMaker Canvas.
Follow these steps:
1. In AWS S3 Console, make sure you are in the correct region.
2. Navigate to the SageMaker provided S3 bucket and locate the object corresponding to the exported dataset .csv file.
3. Select the object and click on *Copy S3 URI*.
4. Replace the placeholder in the cell below with the copied URI above.

In [ ]:
# Cell 6
# Read the exported data (.csv file) from S3 bucket into Pandas dataframe
df_transaction_data = pd.read_csv("INSERT THE S3 URI OF DATASET .CSV FILE")

### Explore the dataset
Let's take a look at the dataset, check the data types and make sure that it is loaded as expected.

In [ ]:
# Cell 7
df_transaction_data.info()

The output above shows there are 116,705 rows and 17 columns in our datasets. All data is numerical as a result of the transformations applied using Data Wrangler in the Amazon SageMaker Canvas. Let's take a look at thr top 5 rows in the dataset.

In [ ]:
# Cell 8
df_transaction_data.head()

Let's check for data inconsistencies. Let's see if there are any empty rows.

In [ ]:
# Cell 9
# This will print if there are any empty rows in the two datasets
print(f"There are {df_transaction_data.isnull().sum().sum()} empty rows in the dataset.")

We can use the `describe` command to generate some statistics about the dataset. This is a very useful comamnd to see distribution of data. You can also use `describe` on a single column of the Dataframe.

In [ ]:
# Cell 10
df_transaction_data.describe()

With this data lets get some more information like

* `.nunique()` gives us the unique values in a series e.g. `[1, 3, 5, 5, 7]` has 4 unique values
* `.isnull()` gives us a boolean result of whether a value at hand is null or not. We would count the number of unique values and then divide it by the total number of values to get percentage of missing elements
* `.value_counts` gives us the frequency of the occuring value. e.g. `[1, 2, 4, 5, 5, 2]` would give us `{1: 1, 2: 2, 4: 1, 5: 2}`. But if you normalize this, it will represent the relative capacity of the value's occurence. Basically how many times the value has occurred divided by the total values. This is a good way to see the variability in the values of this column.
* `dytpe` give us the data type of the series. e.g. `int`, `float` etc.


In [ ]:
# Cell 11
df_transaction_data_stats = []

# loop over all the columns
for col in df_transaction_data.columns:
    
    # append all stats per column to a list
    df_transaction_data_stats.append(
        (
            col,
            df_transaction_data[col].nunique(),
            df_transaction_data[col].isnull().sum() * 100 / df_transaction_data.shape[0],
            df_transaction_data[col].value_counts(normalize=True, dropna=False).values[0] * 100,
            df_transaction_data[col].dtype,
        )
    )

# finally create a new dataframe with all these values
stats_df = pd.DataFrame(
    df_transaction_data_stats,
    columns=["feature", "unique_values", "percent_missing", "percent_largest_category", "datatype"],
)

# sort these values by largest category column (descending)
stats_df.sort_values("percent_largest_category", ascending=False)

There are no empty rows in our dataset and no missing data. So we do not have to do *Data Imputation* (a method for retaining the majority of the dataset's data and information by substituting missing data with a different value).

### Prepare data for training and inference
In this section, we will split the data into train (70% of transactions), validation (20% of transactions), and test (10% of transactions) subsets. The data would be sorted randomly before splitting to avoid any bias. 

NOTE: Since we are not storing features in a feature store for this workshop, let's drop the **event_time** column.

In [ ]:
# Cell 12
df_transaction_data = df_transaction_data.drop('event_time', axis=1)

In [ ]:
# Cell 13
df_transaction_data.info()

In [ ]:
# Cell 14
train_data, validation_data, test_data = np.split(df_transaction_data.sample(frac=1, random_state=1729), [int(0.7 * len(df_transaction_data)), int(0.9 * len(df_transaction_data))])   # Randomly sort the data then split out first 70%, second 20%, and last 10%

Let's review the train data to ensure we have 70% of the dataset.

In [ ]:
# Cell 15
train_data.info()

The training data has 81,693 transactions. Now let's check validation and test datasets.

In [ ]:
# Cell 16
validation_data.info()

In [ ]:
# Cell 17
test_data.info()

The validation and test dataset has 23,341 and 11,671 transactions respectively confirming the splitting as expected. 

Now, for the training algorithm, the target variable is going to be is_fraud_yes which needs to be the first column of the training and validation dataset. Currently, it is the last column in our datasets, so lets move it to the first one. Along with that, we do not need the is_fraud_no column, so let's remove it. The datasets do not need to have either of index and header, so let's remove those as well. We need to keep it for the test data though.

In [ ]:
# Cell 18
pd.concat([train_data['is_fraud_yes'], train_data.drop(['is_fraud_no', 'is_fraud_yes'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['is_fraud_yes'], validation_data.drop(['is_fraud_no', 'is_fraud_yes'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)

In the folder, you can now see two files created **train.csv** and **validation.csv**. Now, let's define a prefix for S3 location to store these files.

In [ ]:
# Cell 19
# S3 bucket for training data - this will create bucket like 'Sagemaker-<region>-<Your AccountId>'
bucket=sess.default_bucket()
prefix = "tech-summit-payments-fraud-workshop"

%store bucket
%store prefix

### Upload the Train and Validation dataset to S3
Let's store these files into a folder with the prefix above under train and validation subfolders.

In [ ]:
# Cell 20
import boto3
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

The files are now uploaded to the S3 bucket. Please verify using AWS S3 Console by navigating to the bucket and **tech-summit-payments-fraud-workshop** directory (check *train* and *validation* directory).

## Module 2: Train and deploy XGBoost model
---
In this section, we will train a ML model using [XGBoost Open Source Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/xgboost.html) model. XGBoost is an extremely popular, open-source package for gradient boosted trees. It is computationally powerful, fully featured, and has been successfully used in many machine learning competitions.

Once the training and test datasets have been persisted in S3, you can start training a model by defining which SageMaker Estimator you'd like to use. This estimator is accessed via the SageMaker SDK, but mirrors the open source version of the [XGBoost Python package](https://xgboost.readthedocs.io/en/latest/python/index.html). Any functionality provided by the XGBoost Python package can be implemented in your training script. This estimator includes reference to the relevant training algorithm from ECR.

In [ ]:
# Cell 21
container = sagemaker.image_uris.retrieve(region=boto3.Session().region_name, framework='xgboost', version='latest')

Let's define th path to the train and validation dataset that was uploaded to S3 bucket.

In [ ]:
# Cell 22
s3_input_train = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

### Training: Create the estimator
If you want to explore the breadth of functionality offered by the SageMaker XGBoost Framework you can read about all the configuration parameters by referencing the inheriting classes. The XGBoost class inherits from the Framework class and Framework inherits from the EstimatorBase class:
* [XGBoost Estimator documentation](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/xgboost.html#sagemaker.xgboost.estimator.XGBoost)
* [Framework documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Framework)
* [EstimatorBase documentation](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)

For this example, we will use the following parameters for the XGBoost estimator:
* `entry_point` - Path to the Python source file which should be executed as the entry point to training.
* `hyperparameters` - Hyperparameters that will be used for training. The hyperparameters are made accessible as a dict[str, str] to the training code on SageMaker.
* `output_path` - S3 location for saving the training result (model artifacts and output files).
* `framework_version` - XGBoost version you want to use for executing your model training code.
* `instance_type` - Type of EC2 instance to use for training.

In [ ]:
# Cell 23
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)


### Set HyperParameters
These are the parameters which will be sent to our training script in order to train the model. Although they are all defined as "hyperparameters" here, they can encompass XGBoost's [Learning Task Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html#learning-task-parameters), [Tree Booster Parameters](https://xgboost.readthedocs.io/en/latest/parameter.html#parameters-for-tree-booster), or any other parameters you'd like to configure for XGBoost.

For this example, we will use the following hyperparameters for the XGBoost algorithm:

* `max_depth` - Controls how deep each tree within the algorithm can be built. Deeper trees can lead to better fit, but are more computationally expensive and can lead to overfitting. There is typically some trade-off in model performance that needs to be explored between a large number of shallow trees and a smaller number of deeper trees.
* `eta` - Controls how aggressive each round of boosting is. Larger values lead to more conservative boosting.
* `gamma` - Minimum loss reduction required to make a further partition on a leaf node of the tree. The larger, the more conservative the algorithm is.
* `subsample` - Subsample ratio of the training instance. Setting it to 0.5 means that XGBoost randomly collects half of the data instances to grow trees. This prevents overfitting.
* `silent` - Verbosity of printing messages.
* `objective` - Specifies the learning task and the corresponding learning objective. Use `binary:logistic` for binary classification task.
* `num_round` - Controls the number of boosting rounds. This is essentially the subsequent models that are trained using the residuals of previous iterations. More rounds should produce a better fit on the training data, but can be computationally expensive or lead to overfitting.

In [ ]:
# Cell 24
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)


### Start the training job
The following code will launch a training job and store the trained model into S3.

**NOTE: The training process will take ~4 minutes**. You can also check from AWS SageMaker console to verify. Click on the left pane and choose **Training jobs**. You will see the job first in *InProgress* state. Wait till the training job completes and the status becomes *Completed*.

In [ ]:
# Cell 25
xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

# Get Training job name
training_job_name = xgb.latest_training_job.job_name
print("Training job name: {}".format(training_job_name))

You can check the training job details in AWS console. Go to SageMaker console, under **Training**, click **Training jobs**. The job should be in *In_Progress* state.

![trainingjob1.png](images/trainingjob1.png)

After **~2-3mins**, the job status should change to **Completed**.

![trainingjob2.png](images/trainingjob2.png)

Congratulations, you successfully trained the XGBoost model!

Note that the output includes the value of `Billable seconds`, which is the amount of time you will be actually charge for.

### Deploy the model
In this step, you will deploy the trained model to a real-time HTTPS endpoint. This process can take **~2-3mins**.

In [ ]:
# Cell 26
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

The exclamation mark at the end of the output indicates that the endpoint is provisioned. To verify in the AWS Console, go to SageMaker console, under **Inference**, click **Endpoints** on the left pane. You will see the endpoint in *InService* state.

![endpoint2.png](images/endpoint2.png)

**Note down the name of the endpoint as we will need it in one of the next steps.**

## Module 3: Predict and evaluate model performance
---
In this step we will reformat the test CSV data, then run the model to create predictions. We will then evaluate model performance using a confusion matrix (a table with actual and predicted results). In this case we are predicting whether transactions were correctly classified as fraud or not.

First we'll need to determine how we pass data into and receive data from our endpoint. Our data is currently stored as NumPy arrays in memory of our notebook instance. To send it in an HTTP POST request, we'll serialize it as a CSV string and then decode the resulting CSV.

**Note:** For inference with CSV format, SageMaker XGBoost requires that the data does NOT include the target variable. So we will remove **is_fraud_no** as well as the target variable **is_fraud_yes** from the test dataset before running the prediction.

In [ ]:
# Cell 27
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()

In [ ]:
# Cell 28
def predict(data, predictor, rows=500 ):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.drop(['is_fraud_no', 'is_fraud_yes'], axis=1).to_numpy(), xgb_predictor)

In [ ]:
# Cell 29
pd.crosstab(index=test_data['is_fraud_yes'], columns=np.round(predictions), rownames=['actuals'], colnames=['predictions'])

The results show that, out of XXX transactions, XXX were correctly identified as fraud (TP) and XXX as not fraud (TN). Whereas, XXX were incorrectly identified as fraud (FP) and XXX as not a fraud (FN). Based on the formula below, the accuracy of our model is 91%. 

Formula: Accuracy = TP+TN / TP+TN+FP+FN = (10356+302)/(10356+302+104+909) = 0.91 = **91%**

**Note:** This matrix may slightly vary depending on the way model gets trained and if you change the parameters.

### Invoke the endpoint to get a fraud score
---
In this section, we will invoke the model endpoint with a sample test transaction to get a fraud score. As a quick way to get a sample payload, we have copied a row from the test.csv dataset (without the first column *is_fraud_yes*). After invoking the endpoint we compare the result with the corresponding value of fraud.

**Note:** Based on the model's performance, we've concluded that score > 0.1 is determined as a fraud. You can either change the threshold depending on your fraud risk tolerance or fine tune the model if you want better accuracy.

e.g. For the payload "100176615,431466,3775,20,66532,39.8182,-95.4567,23,0,98.3,11,12,2022,10,15", the actual value of is_fraud_yes is 1.0 (fraud) and the fraud score is returned as 0.18 (which is >0.1) indicating that it is predicted as a fraud. 

![fraud-yes.png](images/fraud-yes.png)


In [ ]:
# Cell 30
import boto3

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

#payload = b"493456535,445702,11420,2,18846,41.6041,-76.2598,13344,5,17.39,86,278,2022,10,28" #fraud should be returned as yes
#payload = b"527413937,305778,2428,3,95436,38.4923,-122.9042,17586,13,9.42,29,642,2023,0,10"   #fraud should be returned as yes
#payload = b"455331436,350966,613,13,25401,39.4567,-77.9727,153,10,107.85,108,561,2022,11,12" #fraud should be returned as no
payload = b"100176615,431466,3775,20,66532,39.8182,-95.4567,23,0,98.3,11,12,2022,10,15" #fraud should be returned as yes

response = runtime.invoke_endpoint(
    EndpointName="INSERT THE ENDPOINT NAME HERE", # Replace the endpoint with the one created above. e.g. xgboost-2024-09-04-06-18-06-875"
    Body=payload,
    ContentType="text/csv",
)

print(response["Body"].read())

fraud_score = response["Body"].read()


The output above is 0.18. Since this is greater than on our threshold of 0.1, this transaction will be marked as fraudulent with a score of 0.18 or 180/1000. 

## Clean-up

### IMPORTANT: DO NOT delete the endpoint until the end of this workshop. DO NOT run the cell below as it will delete the endpoint which is referred to in a lambda function later in the workshop.

Amazon SageMaker charges you for instances used by your endpoint. So, if no longer in use, it is highly recommended to delete the endpoint to stop incurring any charges. Run the following code to delete the endpoint we created above.

In [ ]:
# Cell 31
xgb_predictor.delete_endpoint(delete_endpoint_config=True)


The endpoint should now be deleted. Verify this by navigating to AWS Console, go to SageMaker console, under **Inference**, click **Endpoints**. The endpoint should no longer be there.

### If you deleted the endpoing by mistake, re-run the steps in [Deploy the model](#deploy-the-model) section i.e. Cell#26. This will create another endpoint. You will need to use the name of the newly provisioned endpoint in the subsequent sections of this workshop.

## Conclusion
In this lab we learned the process of data exploring, data preparation, along with training, and deploying an XGBoost model using Amazon SageMaker built-in algorithm. Our fraud model was trained and validated using a pre-labeled financial payments transaction data. The accuracy of the model was 91% and we tested it with a sample test transactions to get the fraud score which was above the threshold of 0.1 which correctly indicated that the transaction was fraudulent.